# Shipping Forecast Bot Prototype

Preinstall the required packages

In [12]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Define the data class for the forecast

This data class services for the data exchange between the different sources and the report generator.

In [13]:
from dataclasses import dataclass, asdict
from typing import List

@dataclass
class Forecast:
    source_id: int
    source_name: str
    url: str
    publication_time: str
    synoptic_info: str
    warnings: List[dict]
    forecast_details: dict

    def display_info(self):
        return f"Forecast published at {self.publication_time} on {self.source_name}"

    def to_dict(self):
        return asdict(self)

    @classmethod
    def from_dict(cls, data: dict):
        return cls(
            source_id=data["source_id"],
            source_name=data["source_name"],
            url=data["url"],
            publication_time=data["publication_time"],
            synoptic_info=data["synoptic_info"],
            warnings=data["warnings"],
            forecast_details=data["forecast_details"]
        )

    def __eq__(self, other):
        if not isinstance(other, Forecast):
            return False
        return self.to_dict() == other.to_dict()


## Define the report generator class

The report generator class takes a full forecast from a source and list of subscribed areas for generation of a personalized human-readable report.

In [21]:
class ReportGenerator:

    def __init__(self, forecast: Forecast):
        self.forecast = forecast

    def generate_report(self, subscribed_areas) -> str:
        report_lines = [
            f"Issued by {self.forecast.source_name}\n",
            f"Shipping Forecast Publication Time: {self.forecast.publication_time}\n",
            f"General Synoptic Information: {self.forecast.synoptic_info}\n"
        ]

        # Warnings: include warnings only if any of the affected areas contain one of the user's subscribed areas.
        relevant_warnings = []
        for warning in self.forecast.warnings:
            for warning_area in warning.get("areas", []):
                for area in subscribed_areas:
                    if area.lower() in warning_area.lower():
                        relevant_warnings.append(warning)
                        break
                else:
                    continue
                break

        if relevant_warnings:
            report_lines.append("Warnings:")
            for warning in relevant_warnings:
                report_lines.append(f"  Warning Type: {warning.get('warning_type', 'N/A')}")
                report_lines.append("  Affected Areas:")
                for w_area in warning.get("areas", []):
                    report_lines.append(f"    - {w_area}")
                report_lines.append("")  # blank line for readability
        else:
            report_lines.append("No warnings for your subscribed areas.\n")

        # Forecasts for each of the user's areas
        report_lines.append("Forecasts for your subscribed areas:")
        for user_area in subscribed_areas:
            found = False
            for region, forecast in self.forecast.forecast_details.items():
                # Check if the user's area is present in the forecast region name (case-insensitive)
                if user_area.lower() in region.lower():
                    report_lines.append(f"{region}:")
                    report_lines.append(forecast)
                    report_lines.append("")  # add a blank line between regions
                    found = True
            if not found:
                report_lines.append(f"{user_area}: Forecast not found.\n")

        report_lines.append(f"Source: {self.forecast.url}\n")
        return "\n".join(report_lines)

## Define the DWDSource class

It's one of the sources for the forecast data. It fetches the forecast from the German Weather Service (DWD) website.

In [15]:
from bs4 import BeautifulSoup, NavigableString
import re

class DWDSource:

    def __init__(self, url="https://www.dwd.de/EN/ourservices/seewetternordostseeen/seewetternordostsee.html"):
        # Name of the source
        self.id = 1
        self.name = "DWD Marine"
        # URL of the web page with forecasts from the German Weather Service
        self.url = url
        # Send a GET request to fetch the page content
        headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        }
        self.response = requests.get(url, headers=headers)
        # Check if the request was successful
        if self.response.status_code != 200:
            raise ValueError(f"Failed to download the page. Status code: {self.response.status_code}")

    def get_forecast(self) -> Forecast:
        # Get the HTML content
        html_content = self.response.text
        # Parse the full HTML document
        soup = BeautifulSoup(html_content, "html.parser")
        # Find the <pre> tag which contains the bulletin text
        pre_tag = soup.find("pre")
        if not pre_tag:
            raise ValueError("No <pre> tag found in the HTML.")
        # Get the plain text (for publication time, synoptic info, and warnings)
        pre_text = pre_tag.get_text(separator="\n")
        # Also keep the HTML of the pre tag to leverage the bold (<B>) tags for forecast areas.
        pre_html = str(pre_tag)
        pre_soup = BeautifulSoup(pre_html, "html.parser")
        # --- 1. Extract Publication Time ---
        # Look for a date/time pattern like "10.03.2025, 15.36 UTC"
        pub_time_match = re.search(r"(\d{2}\.\d{2}\.\d{4},\s*\d{2}\.\d{2}\s*UTC)", pre_text)
        publication_time = pub_time_match.group(1) if pub_time_match else "Not found"
        # --- 2. Extract General Synoptic Information ---
        # We look for the line after the bold header "General synoptic situation"
        synoptic_info_lines = []
        lines = pre_text.splitlines()
        synoptic_flag = False
        for line in lines:
            if "general synoptic situation" in line.lower():
                synoptic_flag = True
                continue
            if synoptic_flag:
                # Stop if we hit a blank line or a line that likely begins a new section (e.g. warnings)
                if line.strip().lower().startswith("forecast valid") or line.strip().lower().startswith("until"):
                    break
                synoptic_info_lines.append(line.strip())
        synoptic_info = " ".join(synoptic_info_lines)
        # --- 3. Extract Warnings Information (e.g. gales, strong winds) ---
        # The warnings are given in lines that start with "until ... in the following forecast areas ... are expected:"
        warnings = []
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            # Check for a warning header line using a case-insensitive match
            if line.lower().startswith("until"):
                # Persist the valid period and the warning type
                warning_type = line
                if lines[i+1].strip().lower().endswith("expected:"):
                    i += 1
                    line = lines[i].strip()
                    warning_type += " " + line
                if line.lower().endswith("expected:"):
                    # Collect subsequent lines as warning areas until a blank line or another section starts
                    warning_areas = []
                    i += 1
                    while i < len(lines):
                        next_line = lines[i].strip()
                        next_line_lower = next_line.lower()
                        if next_line == "" or next_line_lower.startswith("until") or next_line.startswith("<B>"):
                            break
                        if "utsire" in next_line_lower:
                            next_line = next_line_lower.replace("utsire", "utsira")
                        warning_areas.append(next_line)
                        i += 1

                    warnings.append({
                        "warning_type": warning_type,
                        "areas": warning_areas
                    })
            else:
                i += 1
        # --- 4. Extract Forecast Details for Each Region ---
        # We only consider forecast areas that are marked with bold (<B>) tags,
        # and skip any sections related to the outlook forecast.
        forecast_header = pre_soup.find(lambda tag: tag.name == "b" and "forecast valid until" in tag.get_text().lower())
        forecast_details = {}
        if forecast_header:
            # Iterate over all <b> tags that come after the forecast header.
            for bold_tag in forecast_header.find_all_next("b"):
                bold_text = bold_tag.get_text(strip=True)
                # Skip any forecast section that is part of the outlook
                if "outlook" in bold_text.lower():
                    break
                # Process only forecast areas: they should end with a colon (e.g., "German Bight:")
                if not bold_text.endswith(":"):
                    continue
                region = bold_text[:-1].strip()  # Remove the trailing colon

                # To avoid duplicates, skip if the region is already present.
                if region in forecast_details:
                    continue

                # Collect all following text (from sibling nodes) until the next bold tag is encountered.
                forecast_info = ""
                for sibling in bold_tag.next_siblings:
                    # Stop at the next bold tag, which indicates the start of the next forecast area.
                    if getattr(sibling, "name", None) == "b":
                        break
                    if isinstance(sibling, NavigableString):
                        forecast_info += sibling.strip() + " "
                    else:
                        forecast_info += sibling.get_text(" ", strip=True) + " "
                forecast_details[region] = forecast_info.strip()
        # return results as a Forecast object
        return Forecast(
            source_id=self.id,
            source_name=self.name,
            url=self.url,
            publication_time=publication_time,
            synoptic_info=synoptic_info,
            warnings=warnings,
            forecast_details=forecast_details
        )

## Define Storage class

In [16]:
import requests
from typing import List

class CloudflareD1Storage:
    """
    Tables:
    CREATE TABLE areas ( area_id INTEGER PRIMARY KEY, area_name TEXT NOT NULL, source_id INTEGER NOT NULL );
    CREATE TABLE sources ( source_id INTEGER PRIMARY KEY, source_name TEXT NOT NULL );
    CREATE TABLE subscriptions ( chat_id INTEGER NOT NULL, source_id INTEGER NOT NULL, area_id INTEGER NOT NULL, PRIMARY KEY (chat_id, source_id, area_id) );

    Indexes:
    CREATE INDEX idx_subscriptions_source_chat ON subscriptions(source_id, chat_id);
    CREATE INDEX idx_subscriptions_chat_id ON subscriptions(chat_id);
    CREATE INDEX idx_area_source ON areas(source_id);
    """

    def __init__(self, cloudflare_account: str, cloudflare_database: str, cloudflare_token: str):
        self.token = cloudflare_token
        self.url = f"https://api.cloudflare.com/client/v4/accounts/{cloudflare_account}/d1/database/{cloudflare_database}/query"
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {cloudflare_token}",
        }

    def _sql(self, sql: str):
        response = requests.post(self.url, headers=self.headers, json={"sql": sql})
        if response.status_code != 200:
            raise ValueError(f"Failed to perform the sql request. Status code: {response.status_code}. SQL: {sql}")
        return response.json().get("result")[0].get("results")

    def get_subscriptions(self, source_id: int) -> dict[int, List[int]]:
        response = self._sql(f"""
            SELECT chat_id, GROUP_CONCAT(area_id, ',') AS area_ids
            FROM subscriptions
            WHERE source_id={source_id}
            GROUP BY chat_id;
        """)
        return {int(row["chat_id"]) : [int(id) for id in row["area_ids"].split(",")] for row in response}

    def get_dictionary(self, source_id: int) -> dict:
        response = self._sql(f"SELECT area_id, area_name FROM areas WHERE source_id={source_id};")
        return {int(row["area_id"]): row["area_name"] for row in response}

    def drop_users(self, chat_ids: List[int]):
        sql_ids = ", ".join(chat_ids)
        self._sql(f"DELETE FROM subscriptions WHERE chat_id IN ({sql_ids});")

## Define the ForecastNotifier class

The ForecastNotifier class is responsible for sending the generated report to the users via a messaging service.

In [17]:
class ForecastNotifier:

    def __init__(self, forecast: Forecast, storage: CloudflareD1Storage, tg_token: str):
        self.forecast = forecast
        self.storage = storage
        self.tg_token = tg_token

    def notify(self):
        # Get the subscriptions for the current source
        areas_dict = storage.get_dictionary(self.forecast.source_id)
        user_subscriptions = { chat_id : [areas_dict.get(id) for id in area_ids] for chat_id, area_ids in storage.get_subscriptions(self.forecast.source_id).items() }
        # Generate the report
        report_generator = ReportGenerator(self.forecast)
        failed_chat_ids = []
        for chat_id, subscribed_areas in user_subscriptions.items():
            report = report_generator.generate_report(subscribed_areas)
            telegram_send_url = f"https://api.telegram.org/bot{self.tg_token}/sendMessage?chat_id={chat_id}&text={report}"
            try:
                response = requests.get(telegram_send_url).json()
                if not response["ok"]:
                    failed_chat_ids.append(chat_id)
            except Exception as e:
                print(f"Failed to send message to {chat_id}: {e}")
        # Drop the users with failed notifications
        if len(failed_chat_ids) > 0:
            storage.drop_users(failed_chat_ids)

# Test the pipeline

In [18]:
forecast = DWDSource().get_forecast()
print(forecast)

Forecast(source_id=1, source_name='DWD', url='https://www.dwd.de/EN/ourservices/seewetternordostseeen/seewetternordostsee.html', publication_time='12.03.2025, 16.51 UTC', synoptic_info=' A belt of low-pressure 998 between Barents-Sea, southern Baltic States and central North-Sea with an embedded low 996 Denmark is slowly filling up. At the same time, another low 993 Alpes is deepening, while moving northeast towards Lithuania. A shallow high 1009 North-Scandinavia is continuously shifting east. Another high 1025 Irminger-Sea with a connected ridge 1010 towards Bay of Biscay weakens somewhat.   ', warnings=[{'warning_type': 'until thursday evening in the following forecast areas strong winds are expected:', 'areas': ['german bight', 'fisher', 'viking', 'skagerrak', 'southeastern baltic', 'central baltic']}], forecast_details={'German Bight': 'northwesterly winds 4 to 5, locally 6, at times shower \nsqualls, sea increasing 3 meter.', 'Southwestern North Sea': 'northerly winds 4 to 5, at 

In [19]:
import json

forecast_json = json.dumps(forecast.to_dict())
forecast_from_json = Forecast.from_dict(json.loads(forecast_json))
forecast == forecast_from_json

True

In [22]:
import os

tg_token = os.environ["TOKEN"]
cloudflare_account_id = os.environ.get("CF_ACC_ID")
cloudflare_database_id = os.environ.get("CF_DB_ID")
cloudflare_token_id = os.environ.get("CF_TOKEN")
storage = CloudflareD1Storage(cloudflare_account_id, cloudflare_database_id, cloudflare_token_id)
ForecastNotifier(forecast_from_json, storage, tg_token).notify()